# SocialAL Model
# Fit prior model to data - multiple subjects
KLS 2.17.21  
Project info: https://osf.io/b48n2/

Model modified from :
Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

### Python Version

In [1]:
import sys
print(sys.version)  

3.7.4 (default, Aug 13 2019, 15:17:50) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


### Load modules

In [2]:
import numpy as np
import random
import math
import pandas as pd
from scipy.optimize import minimize
import os
from decimal import *
getcontext().prec = 1000 # increases the precision of data representation 

### Define functions

In [3]:
def update_value(Prob):
    invest = [0,3,6,9]
    retain = [9-x for x in invest] #print ("Retain list is: ", retain)
    shared = [2*x for x in invest] #print ("Shared list is: ", shared)
    EV = [(retain[x] + Prob*shared[x]) for x in range(0,4)]
    return EV

def update_prob(recip, Prob, a_gain, a_loss):
    gain = max(recip - Prob, 0)
    loss = min(recip - Prob, 0)
    Prob = Prob + a_gain * gain + a_loss * loss
    return Prob

def get_action_selection_prob(beta, EV, choice):
    actionProb = Decimal(np.exp(beta*EV[choice-1])/np.sum([np.exp(beta*x) for x in EV]))
    return actionProb

def get_action_selection_probs(beta, EV):
    actionProbs = [get_action_selection_prob(beta, EV, x) for x in range(1,5)]
    return actionProbs

def get_likelihood_action(params, data):
    a_gain = params[0]
    a_loss = params[1]
    beta = params[2]
    prob = params[3:6]
    
    # initialize variables
    #prob = [0.5, 0.5, 0.5] #allowed to vary for best-fit
    #ev = [[9,9,9,9],[9,9,9,9],[9,9,9,9]] #allowed to vary for best-fit prob
    ev = [update_value(prob[0]), update_value(prob[1]), update_value(prob[2])]
    
    totalLLH = 0
    for trial in range(0, len(data)):
        trustee = data['Stim_Sequence'][trial] # get trustee type
        choice = data['Choice'][trial] # get choice made by participant
        response = data['Trustee_Response'][trial] # get response from trustee
        
        # compute the probability of selecting each option for that trustee
        probs = get_action_selection_probs(beta, ev[trustee])
        
        if choice != 0:
            # use the probability of the selection (choice-probability) to update log likelihood
            cprob = probs[choice-1]
        
            #add to cumulative log likelihood
            totalLLH += -math.log(cprob)
        
            # update prob and value
            if choice != 1:
                prob[trustee] = update_prob(response, prob[trustee], a_gain, a_loss)
            ev[trustee] = update_value(prob[trustee])       
    return totalLLH

def model_fit(data):
    
    tries = 10000 #  number of tries to find the best-fit parameter
    lowestLLH = math.inf 
    bestFit = 'NA'
    
    for i in range(tries):
        
        # initialize free parameters with randomly chosen numbers
        a_gain=random.uniform(0, 1)
        a_loss=random.uniform(0, 1)
        beta=random.uniform(0, 1)
        prob1=random.uniform(0,1)
        prob2=random.uniform(0,1)
        prob3=random.uniform(0,1)
        params = [a_gain, a_loss, beta, prob1, prob2, prob3]

        # trying different solvers in the minimize call...
        results = minimize(get_likelihood_action, 
                           params, args =(data), bounds = [(0, 1), (0, 1), (1e-10, 20), (0, 1), (0, 1), (0, 1)], 
                           options = {'maxiter': 10000, 'disp': False})
        if (lowestLLH > results['fun'] and results['success']== True):
            lowestLLH = results['fun']
            bestFit = results
        if bestFit == 'NA':
            line = ['NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA']
        else:
            line = np.append(bestFit['x'], bestFit['fun'])
      
    return line

### Load and clean data

In [4]:
files = os.listdir('../data/modeling')
files.remove('sub-1027.csv')
files.remove('sub-2008.csv')
files.remove('sub-1031.csv')
files.remove('sub-2014.csv')
files.remove('sub-2015.csv')
files.remove('sub-2016.csv')
files.remove('sub-2029.csv')
files.remove('sub-2032.csv')
files.remove('sub-1040.csv')
younger = [f for f in files if "sub-10" in f]
older = [f for f in files if "sub-20" in f]

In [5]:
def load_and_clean(file):
    path = os.path.join('../data/modeling', file)
    dt = pd.read_csv(path)
    # recode trial type into numbers for model
    def stims(trial_type):
        if trial_type == "Trustworthy":
            return 0
        elif trial_type == "Neutral":
            return 1
        elif trial_type == "Untrustworthy":
            return 2
    dt['Stim_Sequence'] = dt['trial_type'].apply(stims)
    # rename response_key to choice
    def choices(response_key):
        if response_key == 'None':
            return 0 
        else:
            return response_key  
    dt['Choice'] = dt['response_key'].apply(choices)
    dt['Choice'] = pd.to_numeric(dt['Choice'])
    # calculte the trustee response
    def resp(trial_earnings):
        if trial_earnings >= 12:
            return 1
        else:
            return 0
    dt['Trustee_Response'] = dt['trial_earnings'].apply(resp)
    data = dt[['Stim_Sequence','Choice', 'Trustee_Response']]
    return(data)

In [6]:
print(getcontext())
#params = [model_fit(load_and_clean(file)) for file in files]
#params = [model_fit(load_and_clean(file)) for file in younger] # change last code chunk
params = [model_fit(load_and_clean(file)) for file in older] # change last code chunk

Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])


In [7]:
params = pd.DataFrame(params)
#params['id'] = [file[:-4] for file in files]
#params['id'] = [file[:-4] for file in younger]
params['id'] = [file[:-4] for file in older]
# rename columns
params = params.rename({0:'alpha_gain', 1:'alpha_loss', 2:'beta', 3:'iProb_trust', 4:'iProb_neut', 5:'iProb_untrust', 6:'-LLH', 'id':'id'}, axis='columns') 
cols = params.columns.tolist()
cols = cols[-1:] + cols[:-1]
params = params[cols]
params

,id,alpha_gain,alpha_loss,beta,iProb_trust,iProb_neut,iProb_untrust,-LLH
0,sub-2013,0.078475,0.018702,4.986177e-01,0.909029,0.904152,3.572880e-01,49.326959
1,sub-2007,0.197858,0.147657,2.777741e-01,0.875419,0.889690,8.470570e-02,48.712833
2,sub-2006,0.037154,0.005527,1.188255e-01,0.546188,0.928942,4.568895e-01,59.421743
3,sub-2012,0.070119,0.653925,4.200265e-01,0.231505,0.159567,3.483403e-04,47.248475
4,sub-2004,0.043619,0.166149,9.670927e-01,0.721199,0.556243,4.536410e-02,39.367345
5,sub-2010,0.012683,0.243425,5.352006e-02,0.409050,0.150848,3.305989e-03,59.464713
6,sub-2038,0.007649,0.002222,8.165262e-01,0.555616,0.548319,5.223464e-01,61.864955
7,sub-2039,0.279504,0.901708,5.003753e-01,1.000000,0.064231,1.599962e-13,33.213270
8,sub-2011,0.978878,0.816562,2.543110e-01,1.000000,0.183371,3.188643e-12,50.334756
9,sub-2005,0.043407,0.066630,2.909442e-02,0.681090,0.704887,1.140685e-01,59.494963


In [8]:
# save parameters in text file
#params.to_csv(path_or_buf = '../output/prior_model_params.csv', index = False)
#params.to_csv(path_or_buf = '../output/prior_model_params_younger.csv', index = False)
params.to_csv(path_or_buf = '../output/prior_model_params_older.csv', index = False)